In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import pandas as pd


#크롤링할 웹 사이트
URL = 'https://comic.naver.com/webtoon/weekday.nhn'
data = requests.get(URL).text #text로만 만들기
soup = BeautifulSoup(data,'html.parser') #html.parser로 html 형식으로 불러옵니다.

###### 1. 웹툰의 제목 찾기

In [2]:
title = soup.find_all('a', {'class' : 'title'}) 
# <a class = "title">인 html소스를 찾아 title에 전부 추가
#찾고 싶은 리스트는 제목,작가,요일,장르,줄거리,플렛폼(나중에 카카오페이지도 포함할 수도 있기 때문에)

id_list = []; title_list = []; author_list = []; day_list = []; genre_list = []; story_list = []; platform_list = [];
num = 0

#### 21.10.05 ~ 현재 플랫폼과 id_list는 제외했습니다.#####

###### 2. Chrome Driver 사용해서 동적 크롤링

In [2]:
driver = webdriver.Chrome(r"C:/chromedriver/chromedriver.exe")
driver.get(URL)

#chrome version 확인하고 재설치 할 수도 있습니다.

In [7]:
for i in range(len(title)):
    sleep(0.5) 
    # 크롤링 중간 중간 텀을 주어 과부하 생기지 않게 사용

    page = driver.find_elements_by_class_name("title")
    page[i].click() 
    # 요일당 첫번째 웹툰 차례로 클릭하기

    sleep(0.5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') 
    # 이동한 페이지 주소 html.parser

    day = soup.find_all('ul', {'class' : 'category_tab'})
    day = day[0].find('li', {'class' : 'on'}).text[0:1] # 요일 수집하기

    t = title[i].text #title문서에서 text만 뽑아오기
    if (t in title_list):  # 요일 두 개 이상이면 요일만 추가함
        day_list[title_list.index(t)] += ', ' + day
        driver.back()
        continue
        #같은 웹툰이기 때문에 다시 진행할 이유가 없음
    
    
    #id_list.append(num) ; num += 1  # id에 추가
    
    title_list.append(t)  # 제목 리스트에 추가
    day_list.append(day) # 요일 리스트에 추가
    
    #platform_list.append('네이버 웹툰') # 플랫폼 리스트에 추가

    author = soup.find_all('h2') # h2태그에 있음
    author = author[1].find('span', {'class' : 'wrt_nm'}).text[8:] # 8번 째부터 작가님 성함
    author_list.append(author) # 작가 컬럼에 추가

    genre = soup.find('span', {'class' : 'genre'}).text # 장르 부분
    a = genre.index(",")
    genre = genre[a+1:].replace(" ","") #앞부분 스토리, 옴니버스, 드라마는 필요하지 않아서 사용하지 않음
    genre_list.append(genre) # 장르에 추가

    story = soup.find_all('p') # 줄거리 수집
    story = str(story[3])
    story = story.replace('<p>', '').replace('</p>', '').replace('<br/>', '\n') # <br>을 개행으로 바꾸기
    story_list.append(story) # 줄거리에 추가 #<br> tag는 특수문자 제거로 바꾸기

    driver.back() # 뒤로 가기

In [8]:
total_data = pd.DataFrame()
#total_data['id'] = id_list
total_data['title'] = title_list
total_data['author'] = author_list
total_data['day'] = day_list
total_data['genre'] = genre_list
total_data['story'] = story_list
#total_data['platform'] = platform_list

###### 3. Result Crawling 

In [9]:
total_data
#크롤링 결과

,title,author,day,genre,story
0,참교육,채용택 / 한가람,월,액션,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!\n&lt;부...
1,신의 탑,SIU,월,판타지,자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년\n그리고 그런 소년을 시험하는 탑
2,뷰티풀 군바리,설이 / 윤성원,월,드라마,'여자도 군대에 간다면?'본격 여자도 군대 가는 만화!
3,소녀의 세계,모랑지,월,드라마,완벽해 보이지만 사실 외로웠던 백조들과 맘씨 착한 오리가 만나\n여러 갈등을 함께 ...
4,팔이피플,매미 / 희세,월,드라마,"&lt;마스크걸&gt;, &lt;위대한 방옥숙&gt; 매미/희세 작가의 신작!\nS..."
...,...,...,...,...,...
454,제타,하지,일,스릴러,극심한 슬럼프에 안드로이드 '제타'의 그림을 표절해버린 금세기 최고의 천재 화가 '...
455,독신마법사 기숙아파트,기르답 / 재윤,일,로맨스,외교부 문관이 되어 상경한 평범한 아가씨 랑세는 어쩔 수 없는 사정으로 독신 마법사...
456,샤인 스타,김현,일,로맨스,전학생 선유별이 좋아하는 김샤인은 학교에서 유명한 달리기 스타. 잘생긴 얼굴에 뛰어...
457,푸른불꽃,닺,일,드라마,유년시절 트라우마를 겪은 '문가야'. 새롭게 전학 온 학교에서 완벽한 모범생이자 모...


In [15]:
total_data.to_csv('Naver_Webtoon_Crawling.csv', encoding = 'utf-8-sig')

In [14]:
total_data

,title,author,day,genre,story
0,참교육,채용택 / 한가람,월,액션,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!\n&lt;부...
1,신의 탑,SIU,월,판타지,자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년\n그리고 그런 소년을 시험하는 탑
2,뷰티풀 군바리,설이 / 윤성원,월,드라마,'여자도 군대에 간다면?'본격 여자도 군대 가는 만화!
3,소녀의 세계,모랑지,월,드라마,완벽해 보이지만 사실 외로웠던 백조들과 맘씨 착한 오리가 만나\n여러 갈등을 함께 ...
4,팔이피플,매미 / 희세,월,드라마,"&lt;마스크걸&gt;, &lt;위대한 방옥숙&gt; 매미/희세 작가의 신작!\nS..."
...,...,...,...,...,...
454,제타,하지,일,스릴러,극심한 슬럼프에 안드로이드 '제타'의 그림을 표절해버린 금세기 최고의 천재 화가 '...
455,독신마법사 기숙아파트,기르답 / 재윤,일,로맨스,외교부 문관이 되어 상경한 평범한 아가씨 랑세는 어쩔 수 없는 사정으로 독신 마법사...
456,샤인 스타,김현,일,로맨스,전학생 선유별이 좋아하는 김샤인은 학교에서 유명한 달리기 스타. 잘생긴 얼굴에 뛰어...
457,푸른불꽃,닺,일,드라마,유년시절 트라우마를 겪은 '문가야'. 새롭게 전학 온 학교에서 완벽한 모범생이자 모...
